In [43]:
class CFPB:
    """This class helps you initialise perform operations on a Python class object called CFPB"""
    def __init__(self, base_url = None, name=None, creationTimeStamp=None,  modifiedTimeStamp=None, createdBy=None, modifiedBy=None, start_date=None, end_date=None, has_narrative=None, dataframes=[], profiles=[]) -> None:
        
        import datetime

        # Initialisation of attributes
        self.base_url = None
        self.name = None
        self.creationTimeStamp = None
        self.modifiedTimeStamp = None
        self.createdBy = None
        self.modifiedBy = None
        self.start_date = datetime.datetime.now()
        self.end_date = datetime.datetime.now()
        self.has_narrative = None
        self.dataframes = []
        self.profiles = [None, None, None, None]

        # Assign attributes based on what's passed

        self.base_url = base_url if base_url else "https://www.consumerfinance.gov/data-research/consumer-complaints/search/api/v1/"
        self.name = name if name else self.name
        self.creationTimeStamp = creationTimeStamp if creationTimeStamp else datetime.datetime.now()
        self.modifiedTimeStamp = modifiedTimeStamp if modifiedTimeStamp else datetime.datetime.now()
        self.createdBy = createdBy if createdBy else self.createdBy
        self.modifiedBy = modifiedBy if modifiedBy else self.modifiedBy
        self.start_date = datetime.datetime.now() if start_date else self.start_date
        self.end_date = datetime.datetime.now() if end_date else self.end_date
        self.has_narrative = has_narrative if has_narrative else self.has_narrative
        self.dataframes = dataframes if dataframes else self.dataframes
        self.profiles = profiles if profiles else self.profiles

    def create_request_url(self,base_url, start_date, end_date, has_narrative):
        """This function creates a request URL based on the parameters passed"""
        request_url = base_url + "?format=json&date_received_min=" + start_date + "&date_received_max=" + end_date + "&has_narrative=" + has_narrative
        return request_url

    def get_data(self,request_url):
        """This function gets data from the request URL"""
        import requests
        response = requests.get(request_url)
        if response.status_code == 200:
            return response.status_code, response.json()
        else:
            return response.status_code, {"error": "An error occurred while fetching data", "status_code": response.status_code, "reason": response.reason}
        
    def convert_data_to_dataframe(self,data):
        """This function converts data to a pandas dataframe"""
        import pandas as pd
        return pd.DataFrame(data)       
    
    def cluster(self):
        """This function clusters data based on the parameters passed"""
        pass
     
    def load(self, start_date=None, end_date=None, has_narrative=None):
        """This function loads data based on the parameters passed"""
        import datetime
        self.start_date = min(self.start_date, datetime.datetime.strptime(start_date, "%Y-%m-%d")) if start_date else self.start_date
        self.end_date = max(self.end_date, datetime.datetime.strptime(end_date, "%Y-%m-%d")) if end_date else self.end_date
        self.has_narrative = has_narrative if has_narrative else self.has_narrative
        request_url = self.create_request_url(self.base_url, start_date, end_date, has_narrative)
        status_code, data = self.get_data(request_url)
        print(status_code)
        if status_code == 200:
            dataframe = self.convert_data_to_dataframe([ob["_source"]for ob in data])
            self.dataframes.append(dataframe)
            print(f"Data frame of {len(data)} rows and {len(dataframe.columns)} columns loaded to CFPB.dataframes[{len(self.dataframes)-1}]")
        else:
            print(data)

    def describe(self, dataframe_indicator=0):
        """This function returns a dataframe profile"""
        profile = self.dataframes[dataframe_indicator].describe().T
        profile["column_name"]=profile.index
        profile["id"]=list(range(0,len(profile.index)))
        profile.set_index("id", inplace=True)
        self.profiles[dataframe_indicator]=profile
        print(f"Profile loaded to CFPB.profiles[{dataframe_indicator}]")
        pass


In [44]:
new_cfpb = CFPB()

In [45]:
new_cfpb.load("2024-12-01", "2024-12-31", "true")

200
Data frame of 64833 rows and 18 columns loaded to CFPB.dataframes[0]


In [38]:
new_cfpb.describe(dataframe_indicator=0)

Profile loaded to CFPB.profiles[0]


In [39]:
new_cfpb.profiles[0]

,count,unique,top,freq,column_name
id,,,,,
0,64833,11,Credit reporting or other personal consumer re...,52346,product
1,64833,38727,My credit reports are inaccurate. These inaccu...,1314,complaint_what_happened
2,64833,51,2024-12-10T12:00:00-05:00,3769,date_sent_to_company
3,64833,84,Incorrect information on your report,28394,issue
4,64833,53,Credit reporting,51997,sub_product
5,64833,5652,XXXXX,2185,zip_code
6,4139,3,Servicemember,2764,tags
7,64833,64833,11067767,1,complaint_id
8,64833,2,Yes,64622,timely


In [6]:
type(profile)

pandas.core.frame.DataFrame

In [7]:
profile

,count,unique,top,freq,column_name
id,,,,,
0,64833,11,Credit reporting or other personal consumer re...,52346,product
1,64833,38727,My credit reports are inaccurate. These inaccu...,1314,complaint_what_happened
2,64833,51,2024-12-10T12:00:00-05:00,3769,date_sent_to_company
3,64833,84,Incorrect information on your report,28394,issue
4,64833,53,Credit reporting,51997,sub_product
5,64833,5652,XXXXX,2185,zip_code
6,4139,3,Servicemember,2764,tags
7,64833,64833,11067767,1,complaint_id
8,64833,2,Yes,64622,timely


In [8]:
from ydata_profiling import ProfileReport

In [9]:
y_profile = ProfileReport(data, title="Profiling Report")

In [10]:
y_profile

Summarize dataset:   0%|          | 0/5 [00:00<?, ?it/s]

Generate report structure:   0%|          | 0/1 [00:00<?, ?it/s]

Render HTML:   0%|          | 0/1 [00:00<?, ?it/s]

In [17]:
profile_transpose.index

Index(['product', 'complaint_what_happened', 'date_sent_to_company', 'issue',
       'sub_product', 'zip_code', 'tags', 'complaint_id', 'timely',
       'consumer_consent_provided', 'company_response', 'submitted_via',
       'company', 'date_received', 'state', 'consumer_disputed',
       'company_public_response', 'sub_issue'],
      dtype='object')

In [30]:
profile_transpose["id"] = list(range(0,len(profile_transpose.index)))

profile_transpose.set_index("id", inplace=True)

In [31]:
profile_transpose

,count,unique,top,freq,ID
id,,,,,
0,64833,11,Credit reporting or other personal consumer re...,52346,product
1,64833,38727,My credit reports are inaccurate. These inaccu...,1314,complaint_what_happened
2,64833,51,2024-12-10T12:00:00-05:00,3769,date_sent_to_company
3,64833,84,Incorrect information on your report,28394,issue
4,64833,53,Credit reporting,51997,sub_product
5,64833,5652,XXXXX,2185,zip_code
6,4139,3,Servicemember,2764,tags
7,64833,64833,11067767,1,complaint_id
8,64833,2,Yes,64622,timely


In [32]:
profile_transpose.index

Index([0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17], dtype='int64', name='id')